In [ ]:
# Lab 4: 민원 처리 보고서 자동 생성 (40분) - 실습 버전
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- Lab 1~3의 모든 함수 통합
- 민원 → 분류 → 검색 → 보고서 자동 생성 파이프라인 구축
- 구조화된 보고서 형식 작성
"""

# ============================================================
# 환경 설정
# ============================================================

import sys, os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = getpass.getpass("API Key: ")
client = OpenAI(api_key=api_key)

print("[완료] 환경 설정 완료!\n")

# ============================================================
# PDF 파일 업로드
# ============================================================

from google.colab import files

print("[파일 업로드] 소방기본법.pdf (10페이지)와 화재예방.pdf (44페이지)를 업로드하세요")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"[완료] {filename} 업로드 완료")

# ============================================================
# Lab 1-3 함수 통합 (이미 구현됨 - 빠른 구축)
# ============================================================

print("\n[진행중] Lab 1-3 함수 재구축...")

# Lab 1: 민원 분류
def classify_complaint(text: str) -> Dict[str, str]:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )
    return {"원문": text, "분류_결과": response.choices[0].message.content}

# Lab 2-3: PDF 처리 및 Vector DB
def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({"page_num": i + 1, "text": text, "char_count": len(text)})
    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    chunks = []
    chunk_id = 0
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1
            start = end - overlap
    return chunks

def get_embedding(text: str) -> List[float]:
    response = client.embeddings.create(model="text-embedding-3-small", input=text)
    return response.data[0].embedding

# ChromaDB 초기화 및 데이터 로드
print("[진행중] Vector DB 구축 중...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages)
fire_law_collection = chroma_client.create_collection("fire_law")
for chunk in fire_law_chunks:
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )

# 화재예방
prevention_pages = extract_text_from_pdf("화재예방.pdf")
prevention_chunks = chunk_text(prevention_pages)
prevention_collection = chroma_client.create_collection("fire_prevention")
for chunk in prevention_chunks:
    embedding = get_embedding(chunk['text'])
    prevention_collection.add(
        ids=[f"prev_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "화재예방가이드"}]
    )

# Lab 3: 검색 함수
def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = fire_law_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = prevention_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    return {
        "법령": search_fire_law(query, n_results_per_source),
        "예방가이드": search_fire_prevention(query, n_results_per_source),
        "total_count": n_results_per_source * 2
    }

print("[완료] 모든 함수 준비 완료!\n")

# ============================================================
# 실습 1: 보고서 생성 파이프라인 설계
# ============================================================

print("="*60)
print("[실습 1] 보고서 생성 파이프라인 이해")
print("="*60)

print("""
민원 처리 보고서 자동 생성 프로세스:

[입력] 민원 내용
   ↓
[1단계] 민원 분류 (Lab 1)
   - classify_complaint() 호출
   - 카테고리, 긴급도, 키워드 추출
   ↓
[2단계] 관련 자료 검색 (Lab 2-3)
   - search_both_sources() 호출
   - 소방기본법에서 법령 검색
   - 화재예방 가이드에서 실무 가이드 검색
   ↓
[3단계] 보고서 생성 (Lab 4)
   - 검색 결과를 컨텍스트로 활용
   - LLM이 공식 보고서 형식으로 작성
   ↓
[출력] 구조화된 민원 처리 보고서
""")

# ============================================================
# 실습 2: 보고서 생성 함수 구현
# ============================================================

print("\n" + "="*60)
print("[실습 2] 보고서 생성 함수 구현")
print("="*60)

def generate_complaint_report(complaint_text: str) -> Dict:
    """
    민원 → 분류 → 검색 → 보고서 자동 생성
    
    TODO: Lab 1-3의 모든 함수를 활용하여 보고서 생성 파이프라인 구축
    
    작업 순서:
    1. classify_complaint()로 민원 분류
    2. search_both_sources()로 관련 자료 검색
    3. 검색 결과를 컨텍스트로 구성
    4. LLM에게 보고서 작성 요청
    
    힌트:
    - 분류 결과와 검색 결과를 모두 프롬프트에 포함
    - system 메시지에서 보고서 형식 명확히 지정
    - 구조화된 딕셔너리로 반환
    """
    
    print("\n[진행중] 보고서 생성 파이프라인 실행...")
    
    # Step 1: 민원 분류 (Lab 1)
    print("  [1/3] 민원 분류 중...")
    # TODO: classify_complaint() 호출
    classification = {}  # classify_complaint(complaint_text)로 변경
    
    if classification:
        print(f"       완료: {classification.get('분류_결과', '').split(chr(10))[0] if classification.get('분류_결과') else '분류 완료'}")
    
    # Step 2: 관련 자료 검색 (Lab 2-3)
    print("  [2/3] 관련 법령 및 가이드 검색 중...")
    # TODO: search_both_sources() 호출
    search_results = {}  # search_both_sources(complaint_text, n_results_per_source=2)로 변경
    
    if search_results:
        print(f"       완료: 법령 {len(search_results.get('법령', []))}개, 가이드 {len(search_results.get('예방가이드', []))}개 검색")
    
    # Step 3: 보고서 본문 생성
    print("  [3/3] 보고서 작성 중...")
    
    """
    TODO: 컨텍스트 구성하기
    
    힌트:
    - law_context = "\n".join([f"- {r['source']} {r['page']}p: {r['text'][:200]}" for r in search_results['법령']])
    - prevention_context = "\n".join([...]) # 동일한 방식
    """
    law_context = ""
    prevention_context = ""
    
    """
    TODO: LLM으로 보고서 생성하기
    
    system 메시지 형식:
    "당신은 소방청 민원 처리 담당자입니다.
     다음 형식으로 공식 보고서를 작성하세요:
     
     【민원 처리 보고서】
     
     1. 민원 내용
     2. 분류 결과
     3. 관련 법령 및 근거
     4. 처리 방안
     5. 예방 가이드
     6. 담당 부서 및 처리 기한"
    
    user 메시지에 포함:
    - [민원 내용] complaint_text
    - [분류 정보] classification['분류_결과']
    - [참고 - 소방기본법] law_context
    - [참고 - 화재예방 가이드] prevention_context
    """
    
    report = ""  # LLM 응답으로 채우세요
    
    if report:
        print("       완료: 보고서 생성 완료!")
    else:
        print("       [아직 구현되지 않았습니다]")
    
    return {
        "민원원문": complaint_text,
        "분류": classification,
        "법령검색": search_results.get('법령', []),
        "예방검색": search_results.get('예방가이드', []),
        "보고서": report
    }

# ============================================================
# 실습 3: 보고서 생성 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 보고서 생성 테스트")
print("="*60)

# 테스트 민원 1
test_complaint_1 = "상가 건물 복도에 소화기가 없고, 비상구도 짐으로 막혀있습니다. 안전 점검을 요청합니다."

print(f"\n[테스트 1]")
print(f"민원: {test_complaint_1}")
print("="*60)

result_1 = generate_complaint_report(test_complaint_1)

if result_1.get('보고서'):
    print("\n[생성된 보고서]")
    print(result_1['보고서'])
else:
    print("\n[아직 구현되지 않았습니다]")

# ============================================================
# 실습 4: 다양한 민원 유형 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] 다양한 민원 유형 테스트")
print("="*60)

test_complaints = [
    "음식점을 새로 열려고 하는데 소방시설을 어떻게 설치해야 하나요?",
    "아파트 지하 주차장에 소방차 전용구역에 차가 주차되어 있어 신고합니다.",
]

for i, complaint in enumerate(test_complaints, 2):
    print(f"\n\n{'='*60}")
    print(f"[테스트 {i}]")
    print(f"민원: {complaint}")
    print("="*60)
    
    result = generate_complaint_report(complaint)
    
    if result.get('보고서'):
        print("\n[생성된 보고서]")
        print(result['보고서'])
    else:
        print("\n[아직 구현되지 않았습니다]")
    
    print("\n" + "="*60)

# ============================================================
# 실습 5: 보고서 품질 개선
# ============================================================

print("\n\n" + "="*60)
print("[실습 5] 보고서 품질 개선")
print("="*60)

print("""
보고서 품질 체크리스트:

[ ] 1. 민원 내용이 정확히 요약되었는가?
[ ] 2. 분류(카테고리/긴급도)가 적절한가?
[ ] 3. 관련 법령이 정확히 인용되었는가?
[ ] 4. 처리 방안이 구체적이고 실행 가능한가?
[ ] 5. 예방 가이드가 실무에 도움이 되는가?
[ ] 6. 담당 부서와 처리 기한이 합리적인가?
[ ] 7. 전체적인 문장이 공식적이고 명확한가?

개선 방법 (직접 실험해보세요):

1. Temperature 조정
   - 현재: 0.4
   - 더 일관된 답변: 0.2
   - 더 창의적: 0.6

2. 프롬프트 개선
   - system 메시지에 더 구체적인 지침 추가
   - 예시 보고서 형식 제시
   - 금지 사항 명시 (예: "추측 금지")

3. 검색 결과 개수 조정
   - 현재: n_results_per_source=2
   - 더 많은 컨텍스트: 3 또는 4
   - 단, 토큰 제한 고려

4. 후처리 추가
   - 생성된 보고서 검증
   - 필수 섹션 누락 체크
   - 포맷팅 개선
""")

# ============================================================
# Lab 4 완료
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 4 완료!")
print("="*60)
print("""
학습 내용:
1. Lab 1-3 함수 통합
2. 민원 처리 파이프라인 구축
3. 구조화된 보고서 자동 생성
4. 다양한 민원 유형 테스트
5. 보고서 품질 분석 및 개선

핵심 성과:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 완전 자동화된 민원 처리 시스템 구축
- 법령 + 실무 가이드 융합 보고서

핵심 함수:
- generate_complaint_report(): 통합 보고서 생성 (Lab 5에서 UI 연결)

다음 Lab:
- Lab 5에서는 이 시스템에 Gradio UI를 추가하여
  누구나 쉽게 사용할 수 있는 웹 인터페이스를 만듭니다.

실습 체크리스트:
[ ] 실습 2: generate_complaint_report 파이프라인 구현
[ ] 실습 3-4: 다양한 민원으로 테스트
[ ] 실습 5: 보고서 품질 개선 실험
""")